In [1]:
import librosa
import numpy as np
from numpy import newaxis
import os

In [2]:
file_dir = os.getcwd()

def getDatafile(file_dir):

    test_list = []
    test_label = [] 
    valide_list = []
    valide_label = []
    train_list = []
    train_label = []
    #定义存放各类别数据和对应标签的列表，列表名对应你所需要分类的列别名

    file_list = []   #liste des files de wav
    dic_list = [] # dictionnaire de base des données
    path_list=os.listdir(file_dir)

    for wav_dir in path_list:
        name = wav_dir.split(sep='.')
        if(len(name) == 1) :
            for wav in os.listdir(file_dir+'/'+wav_dir): 
                name_wav = wav.split(sep='.')
                if(len(name_wav) == 2 and name_wav[1] == 'wav' ):
                    file_list.append(wav_dir+'/'+wav)
                    if not(wav_dir in dic_list):
                        dic_list.append(wav_dir)

    print(len(file_list))
    print(dic_list)

    index = {}
    for i in range(len(dic_list)):
        index[dic_list[i]] = i
    #Creates the reverse table that maps labels names to their index

    print(index)
    
    
    #Creates the test_list
    for line in open('testing_list.txt','r'):
        test_list.append(line.strip('\n'))

    perm = np.random.permutation(len(test_list))
    test = list()
    for i in range(len(perm)):
        test.append(test_list[perm[i]])
        label = test_list[perm[i]].strip('\n').split(sep='/')[0]
        test_label.append(index[label])


    #Creates the valide_list
    for line in open('validation_list.txt','r'):
        valide_list.append(line.strip('\n'))
    perm = np.random.permutation(len(valide_list))
    valide = list()
    for i in range(len(perm)):
        valide.append(valide_list[perm[i]])
        label = valide_list[perm[i]].strip('\n').split(sep='/')[0]
        valide_label.append(index[label])


    #Creates the train_list
    for line in file_list:
        if not (line in test_list ):
            if not (line in valide_list):
                train_list.append(line)
    perm = np.random.permutation(len(train_list))
    train = list()
    for i in range(len(perm)):
        train.append(train_list[perm[i]])
        label = train_list[perm[i]].split(sep='/')[0]
        train_label.append(index[label])
        
        
    return index, train, train_label, valide, valide_label, test, test_label

In [3]:
index, train, train_label, valide, valide_label, test, test_label = getDatafile(file_dir)
    

105829
['right', 'eight', 'cat', 'tree', 'backward', 'learn', 'bed', 'happy', 'go', 'dog', 'no', 'wow', 'follow', 'nine', 'left', 'stop', 'three', 'sheila', 'one', 'bird', 'zero', 'seven', 'up', 'visual', 'marvin', 'two', 'house', 'down', 'six', 'yes', 'on', 'five', 'forward', 'off', 'four']
{'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'backward': 4, 'learn': 5, 'bed': 6, 'happy': 7, 'go': 8, 'dog': 9, 'no': 10, 'wow': 11, 'follow': 12, 'nine': 13, 'left': 14, 'stop': 15, 'three': 16, 'sheila': 17, 'one': 18, 'bird': 19, 'zero': 20, 'seven': 21, 'up': 22, 'visual': 23, 'marvin': 24, 'two': 25, 'house': 26, 'down': 27, 'six': 28, 'yes': 29, 'on': 30, 'five': 31, 'forward': 32, 'off': 33, 'four': 34}


In [4]:
dataset = []

nn_train = len(train)
nx = 128
ny = 32
dataset  = np.zeros((nn_train,1,nx,ny))

i = 0 
for file in train :
    y, sr = librosa.load(file, sr=None)
    melspec = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
    logmelspec = librosa.power_to_db(melspec)
    if logmelspec.size < 4096:
        t=4096-logmelspec.size
        a=np.mean(logmelspec)
        z=np.zeros(t)
        z.fill(a)
#         print(tmp.shape)
        logmelspec=(np.append(logmelspec,z))
    elif logmelspec.size > 4096:
        logmelspec=logmelspec[:4096]
        
    #print(len(logmelspec))
        
    if(len(logmelspec) != 128):    
        logmelspec = np.reshape(logmelspec, (128, 32)) 
    
    logmelspec = (logmelspec - np.min(logmelspec) )/(np.max(logmelspec)  - np.min(logmelspec))
    
    logmelspec = logmelspec[newaxis, :, :]
    #dataset.append(logmelspec)
    dataset[i] = logmelspec
    if( i%1000 == 0):
            print(i,file)
            print(train_label[i])
    i+=1

0 happy/890cc926_nohash_0.wav
7
1000 five/728be664_nohash_0.wav
31
2000 dog/d98f6043_nohash_1.wav
9
3000 stop/b959cd0c_nohash_2.wav
15
4000 two/83c9e7e6_nohash_0.wav
25
5000 stop/a8688b67_nohash_0.wav
15
6000 backward/13dce503_nohash_2.wav
4
7000 one/3402e488_nohash_2.wav
18
8000 eight/8e05039f_nohash_1.wav
1
9000 tree/7106d229_nohash_0.wav
3
10000 zero/332d33b1_nohash_0.wav
20
11000 bed/ec989d6d_nohash_0.wav
6
12000 two/2039b9c1_nohash_1.wav
25
13000 left/3589bc72_nohash_0.wav
14
14000 learn/02ade946_nohash_1.wav
5
15000 cat/31601aad_nohash_0.wav
2
16000 five/617de221_nohash_4.wav
31
17000 three/499be02e_nohash_1.wav
16
18000 one/8aa35b0c_nohash_0.wav
18
19000 five/87d5e978_nohash_1.wav
31
20000 nine/9e42ae25_nohash_2.wav
13
21000 down/627c0bec_nohash_2.wav
27
22000 stop/f42e234b_nohash_0.wav
15
23000 two/b84f83d2_nohash_0.wav
25
24000 one/1c84a139_nohash_0.wav
18
25000 sheila/c50f55b8_nohash_4.wav
17
26000 six/dae01802_nohash_3.wav
28
27000 two/dae01802_nohash_2.wav
25
28000 stop/256

In [5]:
testset = []

nn_test = len(test)
nx = 128
ny = 32
testset  = np.zeros((nn_test,1,nx,ny))

i = 0 
for file in test :
    y, sr = librosa.load(file, sr=None)
    melspec = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
    logmelspec = librosa.power_to_db(melspec)
    if logmelspec.size < 4096:
        t=4096-logmelspec.size
        a=np.mean(logmelspec)
        z=np.zeros(t)
        z.fill(a)
#         print(tmp.shape)
        logmelspec=(np.append(logmelspec,z))
    elif logmelspec.size > 4096:
        logmelspec=logmelspec[:4096]
        
    if(len(logmelspec) != 128):    
          logmelspec = np.reshape(logmelspec, (128, 32)) 
    
    logmelspec = (logmelspec - np.min(logmelspec) )/(np.max(logmelspec)  - np.min(logmelspec))
    #print(logmelspec)
    logmelspec = logmelspec[newaxis, :, :]
    #dataset.append(logmelspec)
    testset[i] = logmelspec
    if( i%1000 == 0):
            print(i,file)
            print(test_label[i])
    i+=1
    
    #if(i>11000): print(file)

0 one/c518d1b1_nohash_1.wav
18
1000 three/94de6a6a_nohash_1.wav
16
2000 off/91b03183_nohash_3.wav
33
3000 nine/ea356919_nohash_2.wav
13
4000 bed/fa446c16_nohash_0.wav
6
5000 two/893705bb_nohash_7.wav
25
6000 seven/9a69672b_nohash_2.wav
21
7000 one/bfd26d6b_nohash_0.wav
18
8000 six/bfd26d6b_nohash_0.wav
28
9000 no/bb05582b_nohash_3.wav
10
10000 nine/1cb788bc_nohash_0.wav
13
11000 house/370844f7_nohash_1.wav
26


In [6]:
import torch
import torch.utils.data
from torch.autograd import Variable # torch 中 Variable 模块
import torchvision
import torchvision.transforms as transforms


#train_tensor = transform1(dataset)
train_tensor = torch.tensor(dataset).float()

target_tensor = torch.tensor(train_label)

print(type(train_tensor))

#print(target_tensor[0])
torch_dataset = torch.utils.data.TensorDataset(train_tensor, target_tensor)

trainloader = torch.utils.data.DataLoader(torch_dataset, batch_size=2,
                                          shuffle=True, num_workers=2)


<class 'torch.Tensor'>


In [8]:
test_tensor = torch.tensor(testset).float()
target_test = torch.tensor(test_label)
torch_testset = torch.utils.data.TensorDataset(test_tensor, target_test)
testloader = torch.utils.data.DataLoader(torch_testset, batch_size=2,
                                          shuffle=True, num_workers=2)
print(testloader)

In [9]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,16,7)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 13 * 1, 35)
        #self.fc2 = nn.Linear(256, 256)
        #self.fc3 = nn.Linear(256, 35)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        #print(x[0])
        x = x.view(-1, 64 * 13 * 1)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc1(x)
        return x


net = Net()

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

In [29]:
from torch.autograd import Variable # torch 中 Variable 模块

for epoch in range(6):  # loop over the dataset multiple times  36

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs = Variable(data[0])
        labels = Variable(data[1])
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs)
        # _,predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20000 == 19999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20000))
            running_loss = 0.0

print('Finished Training')

[1, 20000] loss: 0.731
[1, 40000] loss: 0.728
[2, 20000] loss: 0.723
[2, 40000] loss: 0.720
[3, 20000] loss: 0.716
[3, 40000] loss: 0.712
[4, 20000] loss: 0.710
[4, 40000] loss: 0.706
[5, 20000] loss: 0.697
[5, 40000] loss: 0.708
[6, 20000] loss: 0.690
[6, 40000] loss: 0.705
Finished Training


In [26]:
dataiter = iter(testloader)

inputs, labels = dataiter.next()

print(labels)

outputs = net(inputs)

_,predicted = torch.max(outputs, 1)

print(predicted)


tensor([20, 25])
tensor([20, 25])


In [30]:
corret = 0
summ = 0
for i, data in enumerate(trainloader, 0):
    inputs = Variable(data[0])
    labels = Variable(data[1])
    outputs = net(inputs)

    _,predicted = torch.max(outputs, 1)
    
    if (predicted[0] == labels[0] ) : corret += 1
    if(len(predicted) == 2):
        if (predicted[1] == labels[1] ) : corret += 1
        
    summ += 2
    
print(f" train --> correct {corret} : \t{round(corret/summ*100)}%")

 train --> correct 68364 : 	81%


In [31]:
corret = 0
summ = 0
for i, data in enumerate(testloader, 0):
    inputs = Variable(data[0])
    labels = Variable(data[1])
    outputs = net(inputs)

    _,predicted = torch.max(outputs, 1)
    
    loss = criterion(outputs, labels)
    # print statistics
    running_loss += loss.item()
    
    if (predicted[0] == labels[0] ) : corret += 1
    if(len(predicted) == 2):
        if (predicted[1] == labels[1] ) : corret += 1  
        
    summ += 2
    
print(f"test --> correct {corret} : \t{round(corret/summ*100)}%")
print(f"test --> loss  : \t{round(running_loss/i)}")    
    

test --> correct 8527 : 	77%
test --> loss  : 	1


In [32]:
model_file = 'model_30.pth'
torch.save(Net, model_file)
print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')
    

Saved model to model_30.pth. You can run `python evaluate.py --model model_30.pth` to generate the Kaggle formatted csv file



/anaconda3/envs/python3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
